In [140]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# Toeknising the titles 
We will now decapatilise and tokenise(segment) the titles so that we can process them

In [141]:
#importing df of story rows rather that comments 
df = pd.read_csv('output.csv')


In [142]:
df['title'].head

<bound method NDFrame.head of 0          "What May Happen in the Next Hundred Years", f...
1               Getting Started with JavaScript Unit Testing
2          Armstrong, the Django-based and open-source ne...
3                       Why Web Reviewers Make Up Bad Things
4          You Weren't Meant to Have a Boss: The Cliff Notes
                                 ...                        
5351743    Scrapling: Fast, Adaptive Web Scraping for Python
5351744                         Monkeys Predict US Elections
5351745                                                  NaN
5351746                                                  NaN
5351747                                                  NaN
Name: title, Length: 5351748, dtype: object>

In [143]:
df = df.dropna(subset=['title'])

In [144]:
df = df.head(2500)

In [145]:
import pandas as pd
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Set seed for consistent language detection results
DetectorFactory.seed = 0

# Function to check if the title is in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False  # If detection fails, we assume it's not English

# Filter rows where the title is in English
df['title'] = df['title'].str.lower()
df = df[df['title'].apply(is_english)]

# Display the modified DataFrame
print(df)

           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

In [146]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources if you haven't done so already
nltk.download('punkt')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


# Function to tokenize and apply stemming/lemmatization
#def tokenize_and_process(text):
    #if not isinstance(text, str) or not text:  # Check if the text is a valid non-empty string
        #return ''  # Return empty string for non-string or empty values

    # Tokenize the text by whitespace
    #tokens = text.split()
    
    # Apply stemming and lemmatization
    #processed_tokens = [lemmatizer.lemmatize(stemmer.stem(token)) for token in tokens]
    
    #return ' '.join(processed_tokens)  # Return the processed tokens as a single string

# Function to tokenize without stemming/lemmatization
def tokenize_and_process(text):
    if not isinstance(text, str) or not text:  # Check if the text is a valid non-empty string
        return ''  # Return empty string for non-string or empty values

    # Tokenize the text by whitespace
    tokens = text.split()
    
    return ' '.join(tokens)  # Return the tokens as a single string


# Apply the function to create the 'adjusted_title' column
df['title'] = df['title'].apply(tokenize_and_process)

# Display the modified DataFrame
print(df)


           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omare\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [147]:
import pandas as pd
import re

# Function to replace punctuation with special tokens
def preprocess_text(text):
    # Replace specific punctuation with special tokens
    text = text.replace(',', '<COMMA>')  # Replace commas
    text = text.replace('.', '<FULLSTOP>')  # Replace full stops
    text = text.replace('!', '<EXCLAMATION>')  # Replace exclamations
    return text

# Tokenize the text (split into individual words and special tokens)
def tokenize_text(text):
    # Tokenize by splitting on whitespace and keeping punctuation tokens
    return re.findall(r'\S+|<COMMA>|<FULLSTOP>|<EXCLAMATION>', text)

# Preprocess and tokenize the 'title' column
df['title'] = df['title'].apply(preprocess_text)
df['tokens'] = df['title'].apply(tokenize_text)

# Create a vocabulary (a dictionary mapping tokens to unique IDs)
def create_vocabulary(tokens_series):
    # Flatten the list of token lists into a single list
    all_tokens = [token for tokens in tokens_series for token in tokens]
    # Create a unique token list and assign an ID to each
    vocabulary = {token: idx + 1 for idx, token in enumerate(sorted(set(all_tokens)))}
    return vocabulary

# Get vocabulary from the tokens in the DataFrame
vocab = create_vocabulary(df['tokens'])

# Function to convert tokens to token IDs using the vocabulary
def tokens_to_ids(tokens, vocab):
    return [vocab[token] for token in tokens]

# Apply the function to convert tokens to token IDs
df['token_ids'] = df['tokens'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

# Display the DataFrame with token IDs
print(df)


           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

In [148]:
import pandas as pd
import re


# Function to replace all punctuation with special tokens
def preprocess_text(text):
    # Replace various punctuation with special tokens
    replacements = {
        ',': '<COMMA>',
        '.': '<FULLSTOP>',
        '!': '<EXCLAMATION>',
        '?': '<QUESTION>',
        ';': '<SEMICOLON>',
        ':': '<COLON>',
        '(': '<OPENPAREN>',
        ')': '<CLOSEPAREN>',
        '[': '<OPENBRACKET>',
        ']': '<CLOSEBRACKET>',
        '{': '<OPENBRACE>',
        '}': '<CLOSEBRACE>',
        '\'': '<QUOTE>',
        '\"': '<DOUBLEQUOTE>',
        '-': '<DASH>',
        '_': '<UNDERLINE>',
        '…': '<ELLIPSIS>'  # Ellipsis
    }
    
    for punctuation, token in replacements.items():
        text = text.replace(punctuation, token)  # Replace each punctuation
    return text

# Tokenize the text (split into individual words and special tokens)
def tokenize_text(text):
    # Tokenize by splitting on whitespace and keeping punctuation tokens
    return re.findall(r'\S+|<COMMA>|<FULLSTOP>|<EXCLAMATION>|<QUESTION>|<SEMICOLON>|<COLON>|<OPENPAREN>|<CLOSEPAREN>|<OPENBRACKET>|<CLOSEBRACKET>|<OPENBRACE>|<CLOSEBRACE>|<QUOTE>|<DOUBLEQUOTE>|<DASH>|<UNDERLINE>|<ELLIPSIS>', text)

# Preprocess and tokenize the 'title' column
df['title'] = df['title'].apply(preprocess_text)
df['tokens'] = df['title'].apply(tokenize_text)

# Create a vocabulary (a dictionary mapping tokens to unique IDs)
def create_vocabulary(tokens_series):
    # Flatten the list of token lists into a single list
    all_tokens = [token for tokens in tokens_series for token in tokens]
    # Create a unique token list and assign an ID to each
    vocabulary = {token: idx + 1 for idx, token in enumerate(sorted(set(all_tokens)))}
    return vocabulary

# Get vocabulary from the tokens in the DataFrame
vocab = create_vocabulary(df['tokens'])

# Function to convert tokens to token IDs using the vocabulary
def tokens_to_ids(tokens, vocab):
    return [vocab[token] for token in tokens]

# Apply the function to convert tokens to token IDs
df['token_ids'] = df['tokens'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

# Display the DataFrame with token IDs
print(df)


           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

In [149]:

#import pandas as pd
#import sentencepiece as spm


# Step 1: Preprocess the text (optional)
# def preprocess_text(text):
#     # Replace specific punctuation with special tokens
#     text = text.replace(',', '<COMMA>')  # Replace commas
#     text = text.replace('.', '<FULLSTOP>')  # Replace full stops
#     text = text.replace('!', '<EXCLAMATION>')  # Replace exclamations
#     return text

# # Preprocess the 'title' column
# df['title'] = df['title'].apply(preprocess_text)

# # Step 2: Save titles to a file for training SentencePiece
# with open('titles.txt', 'w') as f:
#     for title in df['title']:
#         f.write(title + '\n')

# # Step 3: Train a SentencePiece model
# # Increase vocab_size to at least 100 to avoid the error
# spm.SentencePieceTrainer.train('--input=titles.txt --model_prefix=m --vocab_size=100 --character_coverage=1.0')

# # Step 4: Load the trained SentencePiece model
# sp = spm.SentencePieceProcessor(model_file='m.model')

# # Tokenize the 'title' column using SentencePiece
# df['tokens'] = df['title'].apply(lambda x: sp.encode(x, out_type=str))

# # Create a vocabulary from the tokens
# def create_vocabulary(tokens_series):
#     all_tokens = [token for tokens in tokens_series for token in tokens]
#     vocabulary = {token: idx + 1 for idx, token in enumerate(sorted(set(all_tokens)))}
#     return vocabulary

# # Get vocabulary from the tokens in the DataFrame
# vocab = create_vocabulary(df['tokens'])

# # Function to convert tokens to token IDs using the vocabulary
# def tokens_to_ids(tokens, vocab):
#     return [vocab[token] for token in tokens]

# # Apply the function to convert tokens to token IDs
# df['token_ids'] = df['tokens'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

# # Display the DataFrame with tokens and token IDs
# print(df)


# embedding 

We now need to get the embedding first we will analyze the embedding process for the wikipedia text before 

In [24]:
import pandas as pd
import re
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download the required NLTK resources if you haven't already
nltk.download('punkt')
nltk.download('wordnet')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Function to preprocess text
def preprocess_text(text):
    # Replace specific punctuation with special tokens
    replacements = {
        ',': '<COMMA>',
        '.': '<FULLSTOP>',
        '!': '<EXCLAMATION>',
        '?': '<QUESTION>',
        ';': '<SEMICOLON>',
        ':': '<COLON>',
        '(': '<OPENPAREN>',
        ')': '<CLOSEPAREN>',
        '[': '<OPENBRACKET>',
        ']': '<CLOSEBRACKET>',
        '{': '<OPENBRACE>',
        '}': '<CLOSEBRACE>',
        '\'': '<QUOTE>',
        '\"': '<DOUBLEQUOTE>',
        '-': '<DASH>',
        '_': '<UNDERLINE>',
        '…': '<ELLIPSIS>'  # Ellipsis
    }

    # Replace punctuation with special tokens
    for punctuation, token in replacements.items():
        text = text.replace(punctuation, token)

    # Lowercase the text
    text = text.lower()
    
    return text

# Function to tokenize and apply stemming/lemmatization
#def tokenize_and_process(text):
    # Tokenize the text by whitespace
    tokens = text.split()
    
    # Apply stemming or lemmatization
    processed_tokens = [lemmatizer.lemmatize(stemmer.stem(token)) for token in tokens]
    
    return processed_tokens

# Function to tokenize without stemming/lemmatization
def tokenize_and_process(text):
    # Tokenize the text by whitespace
    tokens = text.split()
    
    return tokens


# Read the text file
file_path = 'text8.txt'  # Replace with your file path
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Preprocess the content
preprocessed_content = preprocess_text(content)

# Tokenize and process the preprocessed text
tokens = tokenize_and_process(preprocessed_content)

# Create sentences for Word2Vec
sentences = [tokens]  # Wrap tokens in a list to form a single sentence

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=20, min_count=1, workers=4, sg=0)

# Save the model
model.save("word2vec.model")



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omare\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\omare\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
sentences

[['anarchism',
  'originated',
  'as',
  'a',
  'term',
  'of',
  'abuse',
  'first',
  'used',
  'against',
  'early',
  'working',
  'class',
  'radicals',
  'including',
  'the',
  'diggers',
  'of',
  'the',
  'english',
  'revolution',
  'and',
  'the',
  'sans',
  'culottes',
  'of',
  'the',
  'french',
  'revolution',
  'whilst',
  'the',
  'term',
  'is',
  'still',
  'used',
  'in',
  'a',
  'pejorative',
  'way',
  'to',
  'describe',
  'any',
  'act',
  'that',
  'used',
  'violent',
  'means',
  'to',
  'destroy',
  'the',
  'organization',
  'of',
  'society',
  'it',
  'has',
  'also',
  'been',
  'taken',
  'up',
  'as',
  'a',
  'positive',
  'label',
  'by',
  'self',
  'defined',
  'anarchists',
  'the',
  'word',
  'anarchism',
  'is',
  'derived',
  'from',
  'the',
  'greek',
  'without',
  'archons',
  'ruler',
  'chief',
  'king',
  'anarchism',
  'as',
  'a',
  'political',
  'philosophy',
  'is',
  'the',
  'belief',
  'that',
  'rulers',
  'are',
  'unnecessa

In [26]:
# Example usage: getting the vector for a word
word_vector = model.wv['diggers']  # Replace 'example' with any word from your corpus

print(word_vector)

[ 2.04544538e-03  1.17671378e-02 -9.20417160e-03  3.56423738e-03
  5.91390254e-03  4.55391593e-04 -1.33600400e-03  1.93768591e-02
 -9.98119544e-03 -9.82130319e-03  5.58623020e-03 -2.78151222e-03
 -4.82429983e-03  5.54024056e-03  3.65259126e-03 -1.65293068e-02
  1.28892651e-02 -2.80174566e-03 -5.44769270e-03 -2.10510939e-02
  1.87369343e-02 -2.97649810e-03  1.20378733e-02 -7.28998566e-03
  7.51349423e-03 -1.46588273e-02 -1.08006562e-03 -1.44705246e-03
  9.86063969e-04 -3.47614102e-03 -2.09716125e-03  5.81514207e-04
 -2.31736712e-03 -2.02433392e-02 -8.18716455e-03  3.28820758e-03
  2.91072880e-03  3.03560076e-03 -3.70341656e-03 -2.17119642e-02
  4.87884413e-03 -1.19712744e-02  6.17305795e-03 -3.89884668e-03
  9.59011819e-03 -8.28463119e-03 -1.52796856e-03  3.96068627e-03
 -2.88477796e-03  9.27669555e-03 -9.71699413e-03 -3.15303123e-03
 -7.50251347e-05 -1.45531306e-03 -9.59664769e-03  2.03806534e-03
 -5.18709561e-03  7.93699466e-04 -1.62811310e-03  1.02730384e-02
  1.59582254e-02 -6.92654

In [27]:
from math import sqrt, pow

def cosine_similarity(vector1: list[float], vector2: list[float]) -> float:
    """Returns the cosine of the angle between two vectors."""
    # the cosine similarity between two vectors is the dot product of the two vectors divided by the magnitude of each vector

    dot_product = 0
    magnitude_vector1 = 0
    magnitude_vector2 = 0

    vector1_length = len(vector1)
    vector2_length = len(vector2)

    if vector1_length > vector2_length:
        # fill vector2 with 0s until it is the same length as vector1 (required for dot product)
        vector2 = vector2 + [0] * (vector1_length - vector2_length)
    elif vector2_length > vector1_length:
        # fill vector1 with 0s until it is the same length as vector2 (required for dot product)
        vector1 = vector1 + [0] * (vector2_length - vector1_length)

    # dot product calculation
    for i in range(len(vector1)):
        dot_product += vector1[i] * vector2[i]

    # vector1 magnitude calculation
    for i in range(len(vector1)):
        magnitude_vector1 += pow(vector1[i], 2)

    # vector2 magnitude calculation
    for i in range(len(vector2)):
        magnitude_vector2 += pow(vector2[i], 2)

    # final magnitude calculation
    magnitude = sqrt(magnitude_vector1) * sqrt(magnitude_vector2)

    # return cosine similarity
    return dot_product / magnitude

In [35]:
vector1 = model.wv['king']
vector2 = model.wv['man']
similarity = cosine_similarity(vector1, vector2)

print("The cosine similarity between vector1 and vector2 is: ", similarity)

AttributeError: 'Word2Vec' object has no attribute 'wv'

In [23]:
import numpy as np
from gensim.models import Word2Vec

# Load the trained model
model = Word2Vec.load("word2vec.model")

# Function to predict a target word using CBOW
def predict_target_word(context_words, model):
    # Filter out words not in the model's vocabulary
    context_vectors = []
    for word in context_words:
        if word in model.wv:
            context_vectors.append(model.wv[word])
        else:
            print(f"Word '{word}' not in vocabulary.")
    
    # Check if we have valid context vectors
    if not context_vectors:
        return None
    
    # Average the context word vectors
    context_vector = np.mean(context_vectors, axis=0)

    # Find the most similar word to the context vector
    similar_words = model.wv.similar_by_vector(context_vector, topn=5)  # Get top 5 similar words

    return similar_words

# Example sentence and context
sentence = "the quick brown fox jumps over the lazy dog"
target_word = "fox"
context_words = ["the", "quick", "brown", "jumps", "over", "the", "lazy", "dog"]

# Predict the target word using the context
predicted_words = predict_target_word(context_words, model)

# Display predicted words
if predicted_words:
    print("Predicted words for context:", predicted_words)
else:
    print("No valid predictions could be made.")


Predicted words for context: [('the', 0.99992436170578), ('autism', 0.9998777508735657), ('of', 0.9998513460159302), ('or', 0.9998496770858765), ('to', 0.9998484253883362)]


# pytorch  


In [29]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Create a mapping from tokens to their indices
token_to_index = {token: idx for idx, token in enumerate(set(tokens))}
index_to_token = {idx: token for token, idx in token_to_index.items()}

# Convert tokens to their corresponding indices
numerical_data = [token_to_index[token] for token in tokens]

# Prepare training samples (you can use a sliding window approach or any method suitable for your task)
window_size = 10  # For context size
X = []
y = []

for i in range(window_size, len(numerical_data) - window_size):
    context = numerical_data[i - window_size:i] + numerical_data[i + 1:i + window_size + 1]
    target = numerical_data[i]
    X.append(context)
    y.append(target)

X = np.array(X)
y = np.array(y)


In [30]:
class Word2VecDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.long), torch.tensor(self.y[idx], dtype=torch.long)

# Create dataset and dataloader
dataset = Word2VecDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [169]:
import torch.nn as nn

class Word2VecModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, context):
        # Get embeddings for context words
        embeds = self.embeddings(context)
        # Average the embeddings
        avg_embed = embeds.mean(dim=1)
        return self.linear(avg_embed)

# Initialize the model
vocab_size = len(token_to_index)
embedding_dim = 100  # Same as Word2Vec vector size
model = Word2VecModel(vocab_size, embedding_dim)


In [171]:
from tqdm import tqdm  # Import tqdm

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Wrap your DataLoader with tqdm to show progress
    with tqdm(total=len(dataloader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for context, target in dataloader:
            optimizer.zero_grad()  # Clear gradients
            output = model(context)  # Forward pass
            loss = criterion(output, target)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights
            
            pbar.update(1)  # Update the progress bar
            pbar.set_postfix(loss=loss.item())  # Optionally display loss value

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')



Epoch 1/10:   0%|          | 1120/531413 [02:23<18:50:14,  7.82batch/s, loss=12]  


KeyboardInterrupt: 

# pytorch attempt two 